In [1]:
 try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [2]:
if IN_COLAB  :
    from google.colab import drive
    drive.mount('/content/drive')

In [3]:
if IN_COLAB  :
    !git clone https://github.com/sithu31296/semantic-segmentation
    %pip install -U gdown
    %pip install -e .
    import gdown
    from pathlib import Path

    ckpt = Path('./checkpoints/pretrained/segformer')
    ckpt.mkdir(exist_ok=True, parents=True)

    url = 'https://drive.google.com/uc?id=1-OmW3xRD3WAbJTzktPC-VMOF5WMsN8XT'
    output = './checkpoints/pretrained/segformer/segformer.b3.ade.pth'

    gdown.download(url, output, quiet=False)

In [4]:
if not(IN_COLAB)  :
    from IPython.display import display, HTML
    display(HTML("<style>.container { width:80% !important; }</style>"))

import torch
from torch import Tensor
import torch.nn as nn
import numpy as np
from torchvision import io
from torchvision import transforms as T
from PIL import Image
import pickle
import torchvision.transforms.functional as F
import matplotlib.pyplot as plt
import time
from torch.utils.data import DataLoader, Dataset, random_split
import gc
from typing import Tuple
from torch.nn import functional as F
from collections import OrderedDict


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
%cd semantic-segmentation

/home/paolo/Downloads/magistrale/1 anno/visiope/visiope/models/semantic-segmentation


In [5]:
test = False
if test:
    class Generator(nn.Module):
        def __init__(self, nc):
            self.nc = nc
            super(Generator, self).__init__()


        def forward(self, embeds):
            ngf = 64
            nz = [[None] *6]*4

            j = 0
            for embed in embeds:
                nz[j][0] = embed.size()[1]
                for i in range(1,6):
                    if i < 4:
                        nz[j][i] = nz[i-1]//2
                    else:
                        nz[j][i] = nz[i-1]//4
                j = j+1
                print(nz)
            self.main = nn.Sequential(
                #reduction of dimensionality To Be Changed in conv... maybe
                # input is Z, going into a convolution
                nn.ConvTranspose2d( nz[6], ngf * 8, 4, 1, 0, bias=False),
                nn.BatchNorm2d(ngf * 8),
                nn.ReLU(True),
                # state size. ``(ngf*8) x 4 x 4``
                nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
                nn.BatchNorm2d(ngf * 4),
                nn.ReLU(True),
                # state size. ``(ngf*4) x 8 x 8``
                nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
                nn.BatchNorm2d(ngf * 2),
                nn.ReLU(True),
                # state size. ``(ngf*2) x 16 x 16``
                nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
                nn.BatchNorm2d(ngf),
                nn.ReLU(True),
                # state size. ``(ngf) x 32 x 32``
                nn.ConvTranspose2d( ngf, self.nc, 4, 2, 1, bias=False),
                nn.Tanh()
                # state size. ``(nc) x 64 x 64``
            )

            return self.main(embed)

In [6]:
class Generator(nn.Module):
    def __init__(self, nz):
        super(Generator, self).__init__()
        ngf = 64
        nc = 1 
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. ``(ngf*8) x 4 x 4``
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. ``(ngf*4) x 8 x 8``
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. ``(ngf*2) x 16 x 16``
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. ``(ngf) x 32 x 32``
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. ``(nc) x 64 x 64``
        )

    def forward(self, input):
        return self.main(input)

In [7]:
class MLP(nn.Module):
    def __init__(self, dim, embed_dim):
        super().__init__()
        self.proj = nn.Linear(dim, embed_dim)

    def forward(self, x: Tensor) -> Tensor:
        x = x.flatten(2).transpose(1, 2)
        x = self.proj(x)
        return x


class ConvModule(nn.Module):
    def __init__(self, c1, c2):
        super().__init__()
        self.conv = nn.Conv2d(c1, c2, 1, bias=False)
        self.bn = nn.BatchNorm2d(c2)        # use SyncBN in original
        self.activate = nn.ReLU(True)

    def forward(self, x: Tensor) -> Tensor:
        return self.activate(self.bn(self.conv(x)))


class SegFormerHead(nn.Module):
    def __init__(self, dims: list, embed_dim: int = 256, num_classes: int = 19):
        super().__init__()
        for i, dim in enumerate(dims):
            self.add_module(f"linear_c{i+1}", MLP(dim, embed_dim))

        self.linear_fuse = ConvModule(embed_dim*4, embed_dim)
        self.linear_pred = nn.Conv2d(embed_dim, num_classes, 1)
        self.dropout = nn.Dropout2d(0.1)

    def forward(self, features: Tuple[Tensor, Tensor, Tensor, Tensor]) -> Tensor:
        B, _, H, W = features[0].shape
        outs = [self.linear_c1(features[0]).permute(0, 2, 1).reshape(B, -1, *features[0].shape[-2:])]

        for i, feature in enumerate(features[1:]):
            cf = eval(f"self.linear_c{i+2}")(feature).permute(0, 2, 1).reshape(B, -1, *feature.shape[-2:])
            outs.append(F.interpolate(cf, size=(H, W), mode='bilinear', align_corners=False))

        seg = self.linear_fuse(torch.cat(outs[::-1], dim=1))
        seg = self.linear_pred(self.dropout(seg))
        return seg

In [8]:
class MLP(nn.Module):
    def __init__(self, dim, embed_dim):
        super().__init__()
        self.proj = nn.Linear(dim, embed_dim)

    def forward(self, x: Tensor) -> Tensor:
        x = x.flatten(2).transpose(1, 2)
        x = self.proj(x)
        return x

class Flatten(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, x):
        batch_size = x.shape[0]
        return x.reshape(batch_size, -1)
    
class Conv2DModule(nn.Module):
    def __init__(self, cin, cout, ks, s):
        super().__init__()
        self.conv = nn.Conv2d(cin, cout, ks, stride=s, bias=False)
        self.bn = nn.BatchNorm2d(cout)        # use SyncBN in original
        self.activate = nn.ReLU(True)

    def forward(self, x: Tensor) -> Tensor:
        return self.activate(self.bn(self.conv(x)))
    
class Conv1DModule(nn.Module):
    def __init__(self, cin, cout, ks, s):
        super().__init__()
        self.conv = nn.Conv1d(cin, cout, ks, stride=s, bias=False)
        self.bn = nn.BatchNorm2d(cout)        # use SyncBN in original
        self.activate = nn.ReLU(True)

    def forward(self, x: Tensor) -> Tensor:
        return self.activate(self.bn(self.conv(x)))
    
class downsample(nn.Module):
    def __init__(self, C, H, W):
        super().__init__()
        self.downsample = nn.Sequential()
        i = 0
        alt = True
        if H*W > 128:
            while(C>4):

                if H*W*C < 500:
                    break
                
                if H//2<5 or W//2<5:
                    print("H", H)
                    print("W", W)
                                     
                    if alt:
                        self.downsample.append(Flatten())
                        C = C*H*W
                        alt = False
                    print("C", C)  
                    self.downsample.append(Conv1DModule(C, C//2, 3, 1))
                    C = C//2
                    i = i+1
                elif H//2<9 or W//2<9:
                    self.downsample.append(Conv2DModule(C, C//2, 3, 1))
                    C = C//2
                    i = i+1
                    H = H-1
                    W = W-1
                else:
                    self.downsample.append(Conv2DModule(C, C//2, 3, 2))
                    C = C//2
                    i = i+1
                    H = H//2
                    W = W//2

        else:
            while(C>16):

                if H*W*C < 500:
                    break
                    
                if H//2<5 or W//2<5:
                    print("H", H)
                    print("W", W)
                    
                    if alt:
                        self.downsample.append(Flatten())
                        C = C*H*W
                        alt = False
                    print("C", C)
                    self.downsample.append(Conv1DModule(C, C//2, 3, 1))
                    C = C//2
                    i = i+1
                elif H//2<9 or W//2<9:
                    self.downsample.append(Conv2DModule(C, C//2, 3, 1))
                    C = C//2
                    i = i+1
                    H = H-1
                    W = W-1
                else:
                    self.downsample.append(Conv2DModule(C, C//2, 3, 2))
                    C = C//2
                    i = i+1
                    H = H//2
                    W = W//2

        
    def forward(self, x: Tensor) -> Tensor:
        return self.downsample(x)
        
class SegFormerHeadGen(nn.Module):
    
    def __init__(self):
        super().__init__()
        #self.G = Generator().to(device)
    
    def forward(self, features: Tuple[Tensor, Tensor, Tensor, Tensor]) -> Tensor:
        downsamplers = []
        for x in features:
            down = downsample(x.shape[1],x.shape[2],x.shape[3]).to(device)
            downsamplers.append(down)
        
        #downsamplers = [downsample(x.shape[0],x.shape[1],x.shape[2]) for x in features]

        images = []
        
        for i, feature in enumerate(features):
            print(downsamplers[i](feature).size())
            #images.append(self.G(downsamplers[i](feature.flatten)))
    
        
        return images
    

In [9]:
from semseg.models.base import BaseModel
from semseg.models.heads import SegFormerHead
#from resDecode import ResDecode


class SegFormerpp(BaseModel):
    def __init__(self, backbone: str = 'MiT-B0', num_classes: int = 19, head: str = 'B0') -> None:
        super().__init__(backbone, num_classes)
        self.num_classes = num_classes
        self.head = head
        self.decode = SegFormerHead(self.backbone.channels, 256 if 'B0' in backbone or 'B1' in backbone else 768, 3)
        self.newDecode = SegFormerHeadGen()
        self.newDecode.to(device)
        self.apply(self._init_weights)


    def forward(self, x: Tensor) -> Tensor:
        y = self.backbone(x)
        #y = self.decode(y)
        #y = F.interpolate(y, size=x.shape[2:], mode='bilinear', align_corners=False)
        #embeds = []
        y = self.newDecode(y)
        '''
        y = self.decode(y)
        self.upsample = nn.Sequential(        
        torch.nn.ConvTranspose2d(y.shape[1],self.num_classes//2,3,stride=2,output_padding=1, padding=1),
        torch.nn.ConvTranspose2d(self.num_classes//2,self.num_classes,3,stride=2,output_padding=1, padding=1),   
                                )
        y = self.upsample(y).to(device)
        '''
        return y


In [10]:
from semseg.models import *
#from segFormerpp import SegFormerpp

model = eval('SegFormerpp')(
    backbone='MiT-B1',
    num_classes=5
)

try:
    model.load_state_dict(torch.load('checkpointcheckpoints/pretrained/segformer/segformer.b3.ade.pth'))
except:
    print("Download a pretrained model's weights from the result table.")
model.eval()
print('Loaded Model')

Download a pretrained model's weights from the result table.
Loaded Model


In [11]:
%cd ../../dataloader/
from load import Ai4MarsData
%cd ../models/semantic-segmentation

/home/paolo/Downloads/magistrale/1 anno/visiope/visiope/dataloader
/home/paolo/Downloads/magistrale/1 anno/visiope/visiope/models/semantic-segmentation


In [12]:
if IN_COLAB  :
    !git clone https://github.com/airoprojects/visiope
    %cd visiope/dataloader/
    from load import Ai4MarsData
    %cd ../..
    with open('/content/drive/MyDrive/Dataset/data_loader.pkl', 'rb') as f:
        data_loader = pickle.load(f)


    data_loader['dataloader'].dataset.setPermuteX((0,3,1,2))
    data_loader['dataloader'].dataset.setDevice(device,0)
    data_loader['dataloader'].dataset.setDevice(device,1)
    #data_loader['dataloader'].dataset.setdevice(device)

    train_loader, test_loader, validation_loader = data_loader['dataloader'].dataset.splitLoader(80,1)

In [13]:
#local
with open('../../dataloader/data_loader.pkl', 'rb') as f:
    data_loader = pickle.load(f)

data_loader['dataloader'].dataset.setPermuteX((0,3,1,2))
#data_loader['dataloader'].dataset.convertion(0)
data_loader['dataloader'].dataset.setDevice(device,0)
data_loader['dataloader'].dataset.setDevice(device,1)
#data_loader['dataloader'].dataset.setdevice(device)

train_loader, test_loader, validation_loader = data_loader['dataloader'].dataset.splitLoader(80,1)


<class 'torch.Tensor'>
<class 'torch.utils.data.dataset.Subset'>


In [14]:
import torch
import numpy as np
import matplotlib.pyplot as plt

import torchvision.transforms.functional as F

def show_image(imgs):
    imgs = imgs.permute(2,0,1)
    if not isinstance(imgs, list):
        imgs = [imgs]
    fig, axs = plt.subplots(ncols=len(imgs), squeeze=False)
    for i, img in enumerate(imgs):
        img = img.detach()
        img = F.to_pil_image(img)
        axs[0, i].imshow(np.asarray(img))
        axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])

'''  
  print(img.device)
  print(img.shape)
  if (img.device != 'cpu'):
    img.to('cpu')
    print('test')
  print(img.device)
  plt.imshow(img)
  plt.show()
'''

"  \n  print(img.device)\n  print(img.shape)\n  if (img.device != 'cpu'):\n    img.to('cpu')\n    print('test')\n  print(img.device)\n  plt.imshow(img)\n  plt.show()\n"

In [15]:
#local
%cd ../../loss/
from trainer_module import trainer
%cd ../models/semantic-segmentation

/home/paolo/Downloads/magistrale/1 anno/visiope/visiope/loss
/home/paolo/Downloads/magistrale/1 anno/visiope/visiope/models/semantic-segmentation


In [16]:
if IN_COLAB  :
    %cd visiope/loss/
    from trainer_module import trainer
    %cd ../..

In [17]:
""" Trainer module for MER-Segmentation """

from datetime import datetime
import torch
if IN_COLAB  :
    # Setup path for custom imports
    import sys
    sys.path.insert(0, '/content/drive/MyDrive/Github/visiope/loss')
    from loss_functions import *

# Main trainer function
def trainer(parameters, multiple_epochs=False, epoch_index=0, tb_writer=0):
    model = parameters['model']
    device = parameters['device']
    model.to(device)
    if multiple_epochs: 
        train_multiple_epoch(parameters,  model, EPOCHS=100)

    else:
        train_one_epoch(parameters, epoch_index, tb_writer, model)
    
# IMPORTANT: FIND OUT ABOUT TB_WRITER
def train_one_epoch(parameters, epoch_index, tb_writer, model):

    # Initialization of training parameters
    
    loss_fn = parameters['loss']
    optimizer = parameters['optimizer']
    training_set = parameters['training']
    device = parameters['device']
    
    # To keep track of the last loss when the function is executed through multiple epochs
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(training_set):
        # Every data instance is an input + label pair
        inputs, labels = data
        #inputs = inputs.permute(0,3,1,2).to(device)
        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        #print(labels.type())
        #print(outputs)
        #labels = labels.type(torch.DoubleTensor)

        #labels = labels.to(device)       

        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()
        
        # IMPORTANT: UNCOMMENT THIS PART AFTER FINDING OUT ABOUT TB_WRITER
        '''
        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(training_set) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.
        '''

    return last_loss


def train_multiple_epoch(parameters, model, EPOCHS=100):

    # Initialization of training parameters
    loss_fn = parameters['loss']
    #optimizer = parameters['optimizer']
    #training_set = parameters['training']
    validation_set = parameters['validation']

    # Initialization of report parameters
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    writer = 0 #SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))
    epoch_number = 0 # just a counter
    best_vloss = 1_000_000.

    for epoch in range(EPOCHS):
        print('EPOCH {}:'.format(epoch_number + 1))

        # Make sure gradient tracking is on, and do a pass over the data
        model.train(True)
        
        start = time.time()
        
        avg_loss = train_one_epoch(parameters, epoch_number, writer, model)
        
        end = time.time()
        print("epoch time: "+str(end-start)+"s")
        
        # We don't need gradients on to do reporting
        model.train(False)

        # Validation loss
        running_vloss = 0.0
        for i, vdata in enumerate(validation_set):
            vinputs, vlabels = vdata
            
            voutputs = model(vinputs)    
            
            vloss = loss_fn(voutputs, vlabels)
            running_vloss += vloss

        avg_vloss = running_vloss / (i + 1)
        print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

        # IMPORTANT: UNCOMMENT THIS PART AFTER FINDING OUT ABOUT TB_WRITER
        '''
        # Log the running loss averaged per batch
        # for both training and validation
        writer.add_scalars('Training vs. Validation Loss',
                        { 'Training' : avg_loss, 'Validation' : avg_vloss },
                        epoch_number + 1)
        writer.flush()
        '''

        '''
        # Track best performance, and save the model's state
        if avg_vloss < best_vloss:
            best_vloss = avg_vloss
            model_path = 'model_{}_{}'.format(timestamp, epoch_number)
            torch.save(model.state_dict(), model_path)
        '''
        #torch.cuda.memory_summary()
        
        torch.cuda.empty_cache()
        gc.collect()
        
        #torch.cuda.memory_summary()
        
        epoch_number += 1

if __name__ == '__main__':

    # This variable needs to be initialized with dummy values/data to test loss integration
    '''
    model = 0
    optimizer = 0
    loss_fn = 0
    training_set = 0 
    validation_set = 0

    parameters = {
                    'model' : model,
                    'loss' : loss_fn,
                    'optimizer' : optimizer,
                    'training' : training_set,
                    'validation': validation_set
    }
    '''


In [18]:
optimizer = torch.optim.Adam(model.parameters())
loss_fn = torch.nn.CrossEntropyLoss().to(device)
training_set = train_loader
validation_set = test_loader

parameters = {
    'model' : model,
    'loss' : loss_fn,
    'optimizer' : optimizer,
    'training' : training_set,
    'validation' : validation_set,
    'device' : device
}
#trainer(parameters)
trainer(parameters,multiple_epochs=True)

EPOCH 1:
H 9
W 9
C 810
H 9
W 9
C 405
H 9
W 9
C 202
H 9
W 9
C 101
H 9
W 9
C 50
H 9
W 9
C 25
H 9
W 9
C 12
H 7
W 7
C 25088
H 7
W 7
C 12544
H 7
W 7
C 6272
H 7
W 7
C 3136
H 7
W 7
C 1568
H 7
W 7
C 784
H 7
W 7
C 392
H 7
W 7
C 196
H 7
W 7
C 98
H 7
W 7
C 49
H 7
W 7
C 24
torch.Size([1, 4, 9, 9])
torch.Size([1, 4, 5, 5])


RuntimeError: Given groups=1, weight of size [405, 810, 3], expected input[1, 1, 160] to have 810 channels, but got 1 channels instead

In [ ]:
from semseg.datasets import *

#model = model.to(device)
predictions = []
start = time.time()
#print('test')
# Use torch.no_grad() to disable gradient computation during testing
with torch.no_grad():

    for images, labels in train_loader:
        # Move the data to the desired device
        #print(images.shape)
        #images = images.permute(0,3,1,2).to(device)
        #images = images.permute(2,0,1).to(device)
        #images = images [None, :, :, :]
        #print(images.shape)
        
        #print(images.shape)
        #labels = labels.to(device)

        # Forward pass to get the predictions
        with torch.inference_mode():
          prediction = model(images)
        #print(prediction)
        prediction = prediction.softmax(1).argmax(1).to(int)
        #prediction = prediction.round().to(int)
        #print(prediction.shape)
        un = prediction.unique()
        #print(un)
        palette = eval('ADE20K').PALETTE.to(device)
        prediction_map = palette[prediction].squeeze().to(torch.uint8)
        #print(type(prediction_map))
        show_image(prediction_map)
        predictions.append(prediction_map)
        
end = time.time()
print(end-start)

In [ ]:
Flatten.__mro__